In [95]:
from scipy.io.wavfile import read
import numpy as np
import soundfile as sf
import sounddevice as sd
import random

data, sr = sf.read('dog.wav', dtype='int16')  
print(sr)
sd.play(data)

44100


In [96]:
print(data)

[[-44 -37]
 [-45 -39]
 [-46 -43]
 ...
 [  0   1]
 [  0   2]
 [  0   0]]


In [90]:
import wave
import struct

def audio_to_binary(filename):
    with wave.open(filename, "rb") as wav_file:
        # Read the binary data from the wave file
        binary_data = wav_file.readframes(wav_file.getnframes())
        # Unpack the binary data and return it as a list of integers
        audio = list(struct.unpack("B"*len(binary_data), binary_data))
        return audio
        #return np.vectorize(np.binary_repr)(audio, 8)


def binary_to_audio(binary_data, filename, sample_rate=44100, sample_width=2, channels=2):
    # Open a wave file for writing
    with wave.open(filename, "w") as wav_file:
        # Set the sample width and number of channels
        wav_file.setsampwidth(sample_width)
        wav_file.setnchannels(channels)
        # Set the sample rate
        wav_file.setframerate(sample_rate)
        # Write the binary data to the wave file
        for byte in binary_data:
            data = struct.pack("B", byte)
            wav_file.writeframes(data)

In [101]:
audio = audio_to_binary("dog.wav")       #max = 255, min=0 list of integers
bin_audio_list =  np.vectorize(np.binary_repr)(audio, 8)

bin_audio_str = ''.join(bin_audio_list)


255 0


In [94]:
#binary_to_audio(audio, 'hello.wav')  

In [ ]:
def DNA(line, l):
    Base_map = {"00":"A", "01":"G", "10":"C", "11":"T"}
    x=""
    temp = [line[i:i+2] for i in range(0, l, 2)]
    for j in range (l//2) :
         x += Base_map[temp[j]]
    
    return x

def revDNA(line, l):
    Code_map = {"A":"00", "G":"01", "C": "10", "T":"11"}
    x = ""
    temp = [line[i:i+1] for i in range(0,l)]
    for j in range(l):
        x += Code_map[temp[j]]
    return x

In [102]:
mapped_array = np.array([DNA(elem, 8) for elem in bin_audio_list])
print(mapped_array)

['TGGA' 'TTTT' 'TGCT' ... 'AAAA' 'AAAA' 'AAAA']


In [104]:
pre_encrypt = ''.join(mapped_array)


In [106]:
#print(pre_encrypt)


In [ ]:
#key = np.array([random.randrange(2) for _ in range(200)])
key = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGT'
key_size = len(key)

In [ ]:
print(revDNA(key, 47))

In [ ]:
bin_key = revDNA(key, 47)

y = int(bin_key[0:17],2) ^ int(bin_audio_str[0:17],2)
print('{0:b}'.format(y))

In [ ]:
a = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGAA'
la = len(a)
b = 'ATTCG'
lb = len(b)
chunk = [a[0+i:lb+i] for i in range(0, la, lb)]
print("chunk = ", chunk)


In [ ]:
def Xor_string(a,b):
    x=''
    for i in range(len(a)):
        if (a[i] == b[i]):
            x+= '0'
        else:
            x+= '1'
    return x


In [ ]:
key_val = revDNA(b, lb)
temp = chunk[len(chunk)-1]
x = Xor_string(key_val[0:len(temp)*2],revDNA(temp, len(temp)))
xorval = [Xor_string(key_val, revDNA(item,lb)) for item in chunk[0:len(chunk)-1]]
xorval.append(x)

print(xorval)

In [ ]:
def mapDNA(xorlist):
    j=0
    l = len(xorlist[0])
    x = []
    for xor in xorlist:
        litem = len(xor)
        for i in range(0, litem-1, 2):
            if(xor[i]=='1' and xor[i+1]=='1'):
                x.append((i+j*l)//2)
        j+=1
    return x


In [111]:
index = mapDNA(xorval)
print(index)


[2, 4, 5, 8, 19, 22, 25, 26, 28, 30, 32, 34, 37, 39, 44, 46, 47]


In [123]:
randlen = [random.randint(1,6) for i in range (len(index))]
print(randlen)
    

[2, 3, 3, 2, 5, 5, 5, 3, 5, 4, 2, 4, 3, 6, 6, 2, 3]


In [126]:
encr_str = []
x=''
for val in randlen:
    for i in range(val*2):
        x += str(random.randint(0,1))
    encr_str.append(DNA(x, len(x)))
    x=''

In [127]:
encr_str

['AA',
 'TCC',
 'GAG',
 'AG',
 'AGCAC',
 'TCAGG',
 'CAGTC',
 'TGT',
 'GTCAT',
 'GCCA',
 'GG',
 'ACGT',
 'TTG',
 'ACAGGA',
 'ACATAG',
 'TT',
 'AAC']

In [ ]:
#binary_to_audio(binary_data, "example3.wav")